# Title: csck700_bsdd_api

### This notebook automates the semantic enrichment of construction asset parameters using the buildingSMART Data Dictionary (bsDD). It processes technical parameters derived from UFGS tables, matches them with IFC 4.3 and ETIM properties and property sets or classes retrieved through the bsDD API, and populates the corresponding URIs, codes, and names in the dataset.  
### The notebook then generates deterministic UUIDv5-based GUIDs for each IFC and ETIM entity to ensure reproducibility and traceability, while creating placeholder entries when a property lacks its related set or class. After verifying the consistency of GUIDs to detect any conflicts, the enriched DataFrames are exported into a single Excel file, with one sheet per asset type.   
### This workflow produces a consistent and reproducible dataset linking UFGS parameters with bsDD concepts, ready for integration into the graph-based BIM model enrichment process.  

# 0. Table of content:

- [1. Settings:](#1.-Settings:)
- [2. Imports:](#2.-Imports:)
- [3. Helper functions:](#3.-Helper-functions:)  
        - [3.1. bsDD API:](#3.1.-bsDD-API:)   
        - [3.2. Fill bsDD datasets columns:](#3.2.-Fill-bsDD-datasets-columns:)   
        - [3.3. Global unique identifier (GUID) generation and missing IFC Pset and ETIM Class placeholders:](#3.3.-Global-unique-identifier-(GUID)-generation-and-missing-IFC-Pset-and-ETIM-Class-placeholders:)       
- [4. Data load:](#4.-Data-load:)  
- [5. bsDD API:](#5.-bsDD-APIn:)  
        - [5.1. Mapping UFGS Technical properties to IFC/bsDD:](#5.1.-Mapping-UFGS-Technical-properties-to-IFC/bsDD:)   
        - [5.2. Mapping UFGS Technical properties to ETIM/bsDD:](#5.2.-Mapping-UFGS-Technical-properties-to-ETIM/bsDD:)   
- [6. Fill bsDD datasets columns:](#6.-Fill-bsDD-datasets-columns:)  
- [7. Global unique identifier (GUID) generation and missing IFC Pset and ETIM Class placeholders:](#7.-Global-unique-identifier-(GUID)-generation-and-missing-IFC-Pset-and-ETIM-Class-placeholders:)    
- [8. Data save:](#8.-Data-save:)      

# 1. Settings:

In [52]:
BASE = "https://api.bsdd.buildingsmart.org"
IFC43 = "https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3"
ETIM = "https://identifier.buildingsmart.org/uri/etim/etim/10.0"
HEADERS = {
    "accept": "application/json",
    "user-agent": "BenoitCapstone/1.0"
}
TIMEOUT = 30
PROJECT_NAMESPACE = uuid.NAMESPACE_URL

# 2. Imports:

In [41]:
# !pip install pandas
import pandas as pd
from pandas import StringDtype
# !pip install openpyxl
import openpyxl
import warnings
warnings.simplefilter("ignore", UserWarning)
import requests
from requests.adapters import HTTPAdapter, Retry
import time
import re
# !pip install rapidfuzz
from rapidfuzz import fuzz
import json
import uuid

# 3. Helper functions:

## 3.1. bsDD API:

### 3.1.1. Requests session:

In [3]:
#robust GET helper with retries/backoff
session = requests.Session()
retry = Retry(
    total=5,                   # up to 5 attempts
    backoff_factor=0.6,        # 0.6, 1.2, 2.4, ...
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("https://", HTTPAdapter(max_retries=retry))

def _get(path, params, timeout=(5, 30)):  # (connect, read) seconds
    r = session.get(
        f"{BASE}{path}",
        params=params,
        headers={"Accept": "application/json", "Accept-Encoding": "gzip, deflate"},
        timeout=timeout,
    )
    r.raise_for_status()
    return r.json()

### 3.1.2. IFC properties - ETIM properties:

**STEP 1 - TARGETS SELECTION:**

In [4]:
def _select_params_from_df(df):
    """
    From UFGS to list of technical parameters to search.
    """
    return list(df["parameter_name_input"])

In [5]:
def _align_to_length(n, maybe_list):
    """
    Pad/trim a list to length n with None.
    """
    if maybe_list is None:
        return [None] * n
    if len(maybe_list) == n:
        return maybe_list
    return (maybe_list + [None] * (n - len(maybe_list)))[:n]

**STEP 2 - bsDD QUERY:**

In [6]:
def _bsd_textsearch_properties(text, dictionary_uri):
    """
    From /api/TextSearch/v2 (Property) to list of candidate property dicts.
    """
    if text is None:
        return []
    q = str(text).strip()
    if not q or q.lower() in {"nan", "none"}:
        return []
    try:
        data = _get(
            "/api/TextSearch/v2",
            params={
                "SearchText": q,
                "DictionaryUris": [dictionary_uri],
                "TypeFilter": "Property",
                "IncludeInactive": True,
                "IncludePreview": True,
            }
        )
        return data.get("properties") or []
    except Exception as e:
        print(f"{q} - TextSearch error: {e}")
        return []

**STEP 3 - USER CHOICE:**

In [7]:
def _print_and_choose(asset_name, topic_text, candidates, label, code_key="code"):
    """
    Show candidates, ask for an index, return normalized {uri, code, name} or None.
    """
    if not candidates:
        print(f"{topic_text} - No {label} candidates found.")
        return None

    print(f"\n{asset_name.upper()} - {topic_text.upper()} - CANDIDATE {label}:")
    for i, c in enumerate(candidates):
        n = c.get("name") or "(no name)"
        d = c.get("description") or ""
        print(f"[{i}] - {n}: ({d}).")
    print(f"[{len(candidates)}] - None / Not applicable.")

    while True:
        try:
            choice = int(input("Select the index of the best match: "))
            if 0 <= choice <= len(candidates):
                if choice == len(candidates):
                    return None
                pick = candidates[choice]
                return {
                    "uri":  pick.get("uri"),
                    "code": pick.get(code_key) or pick.get("code"),
                    "name": pick.get("name"),
                }
            print("Invalid choice. Try again.")
        except ValueError:
            print("Please enter a number.")

**FROM UFGS TO IFC PROPERTIES:**

In [8]:
def map_ufgs_to_ifc(dfs, df_names, IFC43):
    """
    Map UFGS technical parameters to IFC properties (IFC 4.3 bSDD).
    Returns dict[df_name] -> list[{uri, code, name} | None], aligned to parameter_name_input.
    """
    out = {}

    for df_name in df_names:
        df_asset   = dfs[df_name]
        asset_name = " ".join(df_name.split("_"))

        # STEP 1 - TARGETS SELECTION:
        params = _select_params_from_df(df_asset)
        print(params)  # keep your original visibility

        picks = []
        for p in params:
            # STEP 2 - bsDD QUERY:
            candidates = _bsd_textsearch_properties(p, IFC43)
            # STEP 3 - USER CHOICE:
            sel = _print_and_choose(asset_name, str(p), candidates, label="IFC PROPERTIES", code_key="code")
            picks.append(sel)

        out[df_name] = picks

    return out

**FILL IFC GAPS WITH ETIM:**

In [9]:
def select_etim_props_for_ifc_gaps(dfs, df_names, ifc_props_dict):
    """
    Populate ETIM equivalents only where IFC mappings are missing.
    Returns dict[df_name] -> list[{uri, code, name} | None], aligned 1:1 with UFGS parameters.
    """
    etim_out = {}

    for df_name in df_names:
        print(df_name)
        df_asset   = dfs[df_name]
        asset_name = " ".join(df_name.split("_"))

        # STEP 1 - TARGETS SELECTION:
        params = _select_params_from_df(df_asset)
        ifc_aligned = _align_to_length(len(params), ifc_props_dict.get(df_name))
        etim_list   = [None] * len(params)

        for i, p in enumerate(params):
            # skip if IFC exists at this index
            if ifc_aligned[i]:
                print(f"{p} - IFC equivalent property already found.")
                continue

            # STEP 2 - bsDD QUERY:
            candidates = _bsd_textsearch_properties(p, ETIM)

            # STEP 3 - USER CHOICE:
            sel = _print_and_choose(asset_name, str(p), candidates, label="ETIM PROPERTIES", code_key="referenceCode")
            etim_list[i] = sel

        etim_out[df_name] = etim_list

    return etim_out

### 3.1.3. IFC properties sets - ETIM classes:

**STEP 1 - TARGETS SELECTION:**

In [10]:
def _step1_select_targets(props_list):
    """
    Keep alignment; each entry becomes {'name','uri'} or None.
    """
    out = []
    for p in props_list:
        if not p:
            out.append(None)
        else:
            out.append({
                "name": p.get("name") or "(unnamed)",
                "uri":  p.get("uri")  or ""
            })
    return out

**STEP 2 - bsDD QUERY:**

In [11]:
def _step2_query_property_classes(property_uri):
    """
    From /api/Property/Classes/v1 to list of candidates.
    Handles both 'propertyClasses' or 'classes' keys.
    """
    if not property_uri:
        return []
    try:
        data = _get("/api/Property/Classes/v1", params={"PropertyUri": property_uri})
        return data.get("propertyClasses") or data.get("classes") or []
    except Exception as e:
        print(f"{property_uri or '(no uri)'} - Property/Classes error: {e}")
        return []

**STEP 3 - USER CHOICE:**

In [13]:
def _step3_pick_and_pack(asset_name, topic, candidates, mode):
    """
    Print candidates, ask for an index, and return the packed dict or None.
    mode: 
    - 'pset' for IFC Psets returns {'uri','code', 'name'}
    - 'etim' for ETIM class returns {'uri','code','name'}
    """
    if not candidates:
        print(f"{topic} - No candidates found ({mode}).")
        return None

    print(f"\n{asset_name.upper()} - {topic.upper()} - CANDIDATES ({mode.upper()}):")
    for i, c in enumerate(candidates):
        if mode == "pset":
            label = c.get("propertySet") or "(no code)"
            desc  = c.get("description") or ""
            line  = f"{label}: ({desc})."
        else:  # mode == "etim"
            uri  = c.get("uri") or ""
            code = c.get("code") or (uri.rsplit("/", 1)[-1] if uri else "(no code)")
            name = c.get("name") or "(no name)"
            desc = c.get("description") or ""
            line = f"{code} — {name} ({desc})"
        print(f"[{i}] - {line}")
    print(f"[{len(candidates)}] - None / Not applicable.")

    while True:
        try:
            k = int(input("Select the index of the best match: "))
            if 0 <= k <= len(candidates):
                if k == len(candidates):
                    return None
                hit = candidates[k]
                if mode == "pset":
                    return {"uri": hit.get("uri"), "code": hit.get("propertySet"), "name": hit.get("name")}
                else:  # mode == "etim"
                    uri  = hit.get("uri") or ""
                    code = hit.get("code") or (uri.rsplit("/", 1)[-1] if uri else None)
                    return {"uri": uri, "code": code, "name": hit.get("name")}
            print("Invalid choice. Try again.")
        except ValueError:
            print("Please enter a number.")

**IFC PROPERTY SET - ETIM CLASSES COMMON BASIS:**

In [16]:
def _select_related_for_properties(props_dict, mode):
    """
    Shared 3-step flow: filter, bSDD query, user pick.
    mode: 'pset' (IFC Property Sets) or 'etim' (ETIM Classes)
    Returns:
      - mode='pset': dict[df_name] -> list[{uri, code} | None]
      - mode='etim': dict[df_name] -> list[{uri, code, name} | None]
    """
    out = {}
    for df_name, properties in props_dict.items():
        asset_name = " ".join(df_name.split("_"))
        # STEP 1 - FILTERING:
        targets = _step1_select_targets(properties)
        picks = []
        for t in targets:
            if not t:
                picks.append(None); continue
            if not t["uri"]:
                print(f"{t['name']} - Missing Property URI, skipping.")
                picks.append(None); continue

            # STEP 2 - bsDD QUERY:
            candidates = _step2_query_property_classes(t["uri"])
            # STEP 3 - USER CHOICE:
            sel = _step3_pick_and_pack(asset_name, t["name"], candidates, mode=mode)
            picks.append(sel)
        out[df_name] = picks
    return out

**IFC PROPERTY SET:**

In [17]:
def select_psets_for_properties(ifc_props_dict):
    """
    From IFC property to IFC Property Sets.
    """
    return _select_related_for_properties(ifc_props_dict, mode="pset")

**ETIM CLASSES:**

In [18]:
def select_etim_classes_for_properties(etim_props_dict):
    """
    From ETIM property to ETIM Classes.
    """
    return _select_related_for_properties(etim_props_dict, mode="etim")

## 3.2. Fill bsDD datasets columns:

In [19]:
def apply_property_mapping(dfs, df_names, mapping_dict, col_prefix, exists_col=None):
    """
    Fill {col_prefix}_uri / _code / _name in each df from mapping_dict[df_name].
    Overwrites entire columns (None in mapping_dict → <NA> in df).
    """
    uri_col  = f"{col_prefix}_uri"
    code_col = f"{col_prefix}_code"
    name_col = f"{col_prefix}_name"

    for df_name in df_names:
        df   = dfs[df_name]
        data = mapping_dict.get(df_name, [])
        
        if len(data) != len(df):
            raise ValueError(
                f"Length mismatch for '{df_name}': len(data)={len(data)} vs len(df)={len(df)}"
            )

        # Build new series (aligned to row order), using None where no match
        uris  = [d.get("uri")  if d else None for d in data]
        codes = [d.get("code") if d else None for d in data]
        names = [d.get("name") if d else None for d in data]

        # Assign with string-friendly dtype to avoid FutureWarning
        df[uri_col]  = pd.Series(uris,  dtype=StringDtype())
        df[code_col] = pd.Series(codes, dtype=StringDtype())
        df[name_col] = pd.Series(names, dtype=StringDtype())

        # Optional exists flag (boolean), set True where a mapping exists
        if exists_col:
            df[exists_col] = pd.Series([bool(d) for d in data], dtype="boolean")

        dfs[df_name] = df  # write back

## 3.3. Global unique identifier (GUID) generation and missing IFC Pset and ETIM Class placeholders:

In [98]:
def make_guid(*parts, kind="", asset="", row_index=None, namespace=PROJECT_NAMESPACE):
    """
    Always returns a GUID string.
    - Primary: UUIDv5 from non-empty `parts`
    - Fallback: UUIDv5 from deterministic seed: (asset|kind|row_index)
    """
    name = "|".join(str(p) for p in parts if p not in (None, ""))
    if not name:
        name = f"{asset}|{kind}|{row_index}"
    return str(uuid.uuid5(namespace, name))

In [99]:
def ensure_string_columns(df, columns=None):
    """
    Ensure each column in `columns` exists on `df` with pandas 'string' dtype.
    Creates missing columns and casts existing ones if needed. Modifies df in place.
    """
    if columns is None:
        columns = [
            "ifc_property_guid", "ifc_pset_guid",
            "etim_property_guid", "etim_class_guid",
            "ifc_pset_code", "etim_class_code",
        ]

    for col in columns:
        if col not in df.columns:
            df[col] = pd.Series(index=df.index, dtype=StringDtype())
        elif df[col].dtype != "string":
            df[col] = df[col].astype(StringDtype())
    return df

In [100]:
def df_guid_placeholders(dfs2):
    """
    Populate GUID columns for IFC/ETIM properties and sets/classes across a dict of DataFrames.
    For each DataFrame (keyed by asset name), assigns deterministic GUIDs (via make_guid)
    to: ifc_property_guid, ifc_pset_guid, etim_property_guid, etim_class_guid.
    If an IFC property lacks an IFC pset, sets ifc_pset_code="Pset_PlaceHolder" and its GUID;
    if an ETIM property lacks an ETIM class, sets etim_class_code="Class_PlaceHolder" and its GUID.
    Modifies DataFrames in place and returns the updated dict.
    """

    for name, df in dfs2.items():
    
        if df is None or df.empty:
            continue
            
        # Ensure target columns are present and string-typed:
        df = ensure_string_columns(df)

        # Precompute placeholders once per DF:
        PSET_PLACEHOLDER_CODE  = "Pset_PlaceHolder"
        CLASS_PLACEHOLDER_CODE = "Class_PlaceHolder"
    
        pset_placeholder_guid = make_guid(
            PSET_PLACEHOLDER_CODE,
            kind="IFC_PSET", asset=name, row_index="__placeholder__"
        )
        class_placeholder_guid = make_guid(
            CLASS_PLACEHOLDER_CODE,
            kind="ETIM_CLASS", asset=name, row_index="__placeholder__"
        )
            
        for index, row in df.iterrows():
                
            # IFC property:
            if "ifc_property_uri" in row and pd.notna(row["ifc_property_uri"]):
                df.at[index, "ifc_property_guid"] = make_guid(
                    row.get("ifc_property_uri"), row.get("ifc_property_code"),
                    kind="IFC_PROP", asset=name, row_index=index
                )
                # if IFC Pset is empty -> placeholder:
                if ("ifc_pset_uri" not in row) or pd.isna(row["ifc_pset_uri"]) or not str(row["ifc_pset_uri"]).strip():
                    df.at[index, "ifc_pset_code"] = PSET_PLACEHOLDER_CODE
                    df.at[index, "ifc_pset_guid"] = pset_placeholder_guid 
    
            # IFC Property Set
            if "ifc_pset_uri" in row and pd.notna(row["ifc_pset_uri"]):
                df.at[index, "ifc_pset_guid"] = make_guid(
                    row.get("ifc_pset_uri"), row.get("ifc_pset_code"),
                    kind="IFC_PSET", asset=name, row_index=index
                )      
    
            # ETIM Property
            if "etim_property_uri" in row and pd.notna(row["etim_property_uri"]):
                df.at[index, "etim_property_guid"] = make_guid(
                    row.get("etim_property_uri"), row.get("etim_property_code"),
                    kind="ETIM_PROP", asset=name, row_index=index
                )
                # If ETIM class is empty -> placeholder:
                if ("etim_class_uri" not in row) or pd.isna(row["etim_class_uri"]) or not str(row["etim_class_uri"]).strip():
                    df.at[index, "etim_class_code"] = CLASS_PLACEHOLDER_CODE
                    df.at[index, "etim_class_guid"] = class_placeholder_guid
    
            # ETIM Class
            if "etim_class_uri" in row and pd.notna(row["etim_class_uri"]):
                df.at[index, "etim_class_guid"] = make_guid(
                    row.get("etim_class_uri"), row.get("etim_class_code"),
                    kind="ETIM_CLASS", asset=name, row_index=index
                )

    return dfs2        

In [101]:
def guid_check(dfs):
    """
    {df_name: True/False}. True iff each GUID maps to a single (uri,code,name).
    Repeats allowed; placeholders ignored.
    """
    specs = [
        ("ifc_property", None),
        ("ifc_pset", ("ifc_pset_code", "Pset_PlaceHolder")),
        ("etim_property", None),
        ("etim_class", ("etim_class_code", "Class_PlaceHolder")),
    ]
    out = {}
    for name, df in dfs.items():
        if df is None or df.empty:
            out[name] = True; continue
            
        ok, seen = True, {}
        
        for pref, ph in specs:
            g,u,c,n = f"{pref}_guid", f"{pref}_uri", f"{pref}_code", f"{pref}_name"
            if g not in df.columns: continue
            for _, r in df.iterrows():
                v = r.get(g)
                guid = "" if pd.isna(v) else str(v).strip()
                if not guid: continue

                # Check if the row corresponds to a placeholder (Pset or Class):
                is_ph = False
                if ph:
                    pv = r.get(ph[0])
                    is_ph = (not pd.isna(pv)) and (str(pv).strip() == ph[1])
                if is_ph: 
                    # skip placeholder GUIDs from uniqueness check:
                    continue
                    
                # build a unique identity (uri, code, name):    
                ident = tuple("" if pd.isna(r.get(k)) else str(r.get(k)).strip() for k in (u,c,n))
                # if same GUID seen before but with different (uri, code, name) → error:
                if guid in seen and seen[guid] != ident: 
                    ok = False 
                    break
                # otherwise store it:
                seen[guid] = ident
                
            # stop early if inconsistency found:    
            if not ok: 
                break
                
        out[name] = ok

    for k, v in out.items():
        k_split = k.split("_")
        k_text = " ".join(k_split)
        print(f"{k_text.upper()} has no GUID duplicate value: {v}.")
        
    return out

## 3.4. Data save:

In [102]:
def write_multi_df_to_excel(dfs, output_path):
    """
    Write multiple DataFrames to a single Excel file, one sheet per key in `dfs`.
    Parameters:
        dfs (dict): Mapping of sheet names to pandas DataFrames.
        output_path (str): Path to the output .xlsx file.
    Each DataFrame is written to a sheet named after its dict key with index disabled.
    """
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        for df_name, df in dfs.items():
            # Each df will go to a sheet named after df_name
            df.to_excel(writer, sheet_name=df_name, index=False)

# 4. Data load:

In [20]:
file_path = r"..\data\raw\ufgs_authoring.xlsx"

In [21]:
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names
print(sheet_names)

['description', 'dictionary_keys', 'data_validation', 'stone_wall_chimney', 'roof_wood_beam', 'strip_footing', 'beam_shoe']


In [22]:
df_names = sheet_names[3:]

In [23]:
dfs = dict()
for df_name in df_names:
    dfs[df_name] = pd.read_excel(file_path, sheet_name=df_name)

In [24]:
for name, df in dfs.items():
    print(f"\n### Dataframe: {name} ###")
    display(df.head(10))


### Dataframe: stone_wall_chimney ###


,id,asset_type,ifc_class,parameter_name_input,value_raw,operator,value_normalized,unit,target_label,ifc_exists,...,etim_class_code,etim_class_name,etim_class_guid,bsdd_dictionary_key,source_doc,source_clause,value_standard,mapping_confidence,roundtrip_action,notes
0,stone_001,Stone Wall,IfcWall;IfcChimney,compressive strength,≥ 14 MPa (at 28 days minimum for masonry assem...,≥,14,MPa,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 04 20 00,§2.1.1 / §2.1.2 / §3.6.1.2,ASTM C1314 / ASTM C476 / TMS 402,NaN,NaN,NaN
1,stone_002,Stone Wall,IfcWall;IfcChimney,density,lightweight / medium / normal weight (ASTM C90...,enum,"{lightweight, medium, normal}",NaN,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 04 20 00,§2.2.3.2.6,ASTM C90 / ASTM C129,NaN,NaN,NaN
2,stone_003,Stone Wall,IfcWall;IfcChimney,weather exposure,units to include water-repellent admixture for...,=,water-repellent required for exterior use,NaN,Type,NaN,...,NaN,NaN,NaN,NaN,UFGS 04 20 00,§2.2.3.2.5,ASTM C216 / TMS 402,NaN,NaN,NaN
3,stone_004,Stone Wall,IfcWall;IfcChimney,fire rating,equivalent thickness 68–157 mm for 2–4 hour ra...,range,68–157,mm,Type,NaN,...,NaN,NaN,NaN,NaN,UFGS 04 20 00,§2.2.3.5,ACI 216.1,NaN,NaN,NaN
4,stone_005,Stone Wall,IfcWall;IfcChimney,mortar type,ASTM C270 Type M / S / N (as specified for use),enum,"{M, S, N}",NaN,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 04 20 00,§2.5.1,ASTM C270 / ASTM C1714,NaN,NaN,NaN
5,stone_006,Stone Wall,IfcWall;IfcChimney,grout compressive strength,≥ 14 MPa (minimum per ASTM C476 / C1019),≥,14,MPa,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 04 20 00,§2.5.2,ASTM C476 / ASTM C1019,NaN,NaN,NaN
6,stone_007,Stone Wall,IfcWall;IfcChimney,thermal resistance,typical assembly R-value ≈ 0.2–0.3 m²·K/W per ...,range,0.2–0.3,m²·K/W,Type,NaN,...,NaN,NaN,NaN,NaN,UFGS 04 20 00,§2.2.3.2 / §2.2.3.5,ACI 216.1 / ASHRAE 90.1,NaN,NaN,NaN
7,stone_008,Stone Wall,IfcWall;IfcChimney,flexural strength,≥ 7 MP,≥,7,MPa,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 04 20 00,§ 2.2.5,ASTM C880,NaN,NaN,NaN



### Dataframe: roof_wood_beam ###


,id,asset_type,ifc_class,parameter_name_input,value_raw,operator,value_normalized,unit,target_label,ifc_exists,...,etim_class_code,etim_class_name,etim_class_guid,bsdd_dictionary_key,source_doc,source_clause,value_standard,mapping_confidence,roundtrip_action,notes
0,beam_001,Roof Wood Beam,IfcBeam,bending strength,"7,200 – 8,300 kPa (Fb)",range,7.2 – 8.3,MPa,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 06 10 00,§2.2.1,AWC NDS,NaN,NaN,NaN
1,beam_002,Roof Wood Beam,IfcBeam,tensile strength,"4,800 kPa (Ft)",=,4.8,MPa,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 06 10 00,§2.2.1,AWC NDS,NaN,NaN,NaN
2,beam_003,Roof Wood Beam,IfcBeam,compressive strength,"5,400 kPa (Fc)",=,5.4,MPa,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 06 10 00,§2.2.1,AWC NDS,NaN,NaN,NaN
3,beam_004,Roof Wood Beam,IfcBeam,modulus of elasticity,"8,300 MPa (E)",=,"8,300",MPa,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 06 10 00,§2.2.1,AWC NDS,NaN,NaN,NaN
4,beam_005,Roof Wood Beam,IfcBeam,shear strength,"1,550 kPa (Fv)",=,1.55,MPa,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 06 10 00,§2.1.5.1 + §2.2.3,ASTM D2344 / ANSI AITC A190.1,NaN,NaN,NaN
5,beam_007,Roof Wood Beam,IfcBeam,fire rating,Fire-retardant treated; ASTM E84,=,Pass,NaN,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 06 10 00,§1.8 / §2.2,ASTM E84,NaN,NaN,NaN
6,beam_008,Roof Wood Beam,IfcBeam,preservative retention,"CCA, AWPA U1",=,CCA,NaN,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 06 10 00,§1.7 / §2.2,AWPA U1 / T1,NaN,NaN,NaN
7,beam_009,Roof Wood Beam,IfcBeam,durability,"Natural decay-resistant wood – redwood, cedar,...",=,Durable species,NaN,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 06 10 00,§2.1.4,AWPA U1 / EN 350,NaN,NaN,NaN



### Dataframe: strip_footing ###


,id,asset_type,ifc_class,parameter_name_input,value_raw,operator,value_normalized,unit,target_label,ifc_exists,...,etim_class_code,etim_class_name,etim_class_guid,bsdd_dictionary_key,source_doc,source_clause,value_standard,mapping_confidence,roundtrip_action,notes
0,footing_001,Strip Footing,IfcFooting,compressive strength,25 – 35 MPa (@ 28 days),=,30,MPa,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 03 30 01,§ 2.5.2.2 / § 3.14.3.4,ASTM C39,NaN,NaN,NaN
1,footing_003,Strip Footing,IfcFooting,water cement ratio,≤ 0.45,≤,0.45,w/cm,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 03 30 00,§ 2.5.2.2,ACI 211.1 / ACI 211.2,NaN,NaN,NaN
2,footing_004,Strip Footing,IfcFooting,maximum aggregate size,19 – 25 mm,≤,25,mm,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 03 30 00,§ 2.3.3.1,ASTM C33,NaN,NaN,NaN
3,footing_006,Strip Footing,IfcFooting,reinforcement grade,Grade 60 (typ.),=,60,NaN,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 03 30 00,§ 2.6.1,ASTM A615,NaN,NaN,NaN
4,footing_007,Strip Footing,IfcFooting,reinforcement coating,Epoxy-coated / uncoated,=,epoxy / none,NaN,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 03 30 00,§ 2.6.1.2 / § 2.6.1.3 / § 2.6.1.4,ASTM A775 / A934 / A1055 / A955,NaN,NaN,NaN
5,footing_008,Strip Footing,IfcFooting,reinforcement spacing,≤ 150 mm (o.c.),≤,150,mm,Type,NaN,...,NaN,NaN,NaN,NaN,UFGS 03 30 00,§ 3.5.11,ACI 315,NaN,NaN,NaN
6,footing_009,Strip Footing,IfcFooting,concrete cover to reinforcement,≥ 75 mm,≥,75,mm,Type,NaN,...,NaN,NaN,NaN,NaN,UFGS 03 30 00,§ 3.5.12,ACI 318 Tbl 20.6.1.3.1,NaN,NaN,NaN
7,footing_010,Strip Footing,IfcFooting,curing period,≥ 7 days (normal cement),≥,7,days,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 03 30 00,§ 3.13.2,ACI 301 § 5.3,NaN,NaN,NaN



### Dataframe: beam_shoe ###


,id,asset_type,ifc_class,parameter_name_input,value_raw,operator,value_normalized,unit,target_label,ifc_exists,...,etim_class_code,etim_class_name,etim_class_guid,bsdd_dictionary_key,source_doc,source_clause,value_standard,mapping_confidence,roundtrip_action,notes
0,beamshoe_001,Beam Shoe,IfcDiscreetAccessory,material designation,ASTM A36/A36M structural carbon steel,=,A36,NaN,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 05 50 13,§2.2.1,ASTM A36/A36M,NaN,NaN,NaN
1,beamshoe_002,Beam Shoe,IfcDiscreetAccessory,coating type,"Hot-dip galvanized, Z275 (G90)",=,Z275 (G90),NaN,Material,NaN,...,NaN,NaN,NaN,NaN,UFGS 05 50 13,§2.3.1,ASTM A123/A153/A653/A924,NaN,NaN,NaN
2,beamshoe_003,Beam Shoe,IfcDiscreetAccessory,strength grade,Anchor bolts: F1554 Grade 55,=,F1554 Gr 55,ksi,Type,NaN,...,NaN,NaN,NaN,NaN,UFGS 05 50 13,§2.2.8,ASTM F1554,NaN,NaN,NaN
3,beamshoe_004,Beam Shoe,IfcDiscreetAccessory,weld type,Per AWS D1.1/D1.1M – Structural Welding Code (...,=,AWS D1.1/D1.1M,NaN,Type,NaN,...,NaN,NaN,NaN,NaN,UFGS 05 50 13,§3.5,AWS D1.1/D1.1M,NaN,NaN,NaN


# 5. bsDD API:

## 5.1. Mapping UFGS Technical properties to IFC/bsDD:

#### Introduction to the Mapping Methodology:  

This step establishes correspondences between UFGS technical properties and IFC/bSDD definitions to support semantic enrichment of IFC-based graphs. The approach is semi-automated, combining automated retrieval from the buildingSMART Data Dictionary (bSDD) with manual validation.  

While bSDD provides authoritative URIs, names, and definitions, it does not encode applicability rules linking property sets to specific IfcClass. These rules are instead defined in separate IFC Property Set Definition (PSD) templates. For example, Pset_WallCommon applies to IfcWall and IfcWallType, but this link is absent in the bSDD API.  

Accordingly, this work follows a two-stage process:  

- Automated retrieval using /api/TextSearch/v2 and /api/Property/Classes/v1 to obtain candidate IFC properties, their property sets, and URIs.  

- Manual validation to select the semantically most appropriate property and property set for the target IfcClass.  

This process is used to check whether a UFGS property already exists in the IFC 4.3 dictionary. If a match is found, the corresponding IfcProperty and IfcPropertySet (URI, code, and name) are captured for graph construction. If no equivalent exists in IFC, the search continues in the ETIM dictionary as an alternative source.

### 5.1.1. Properties automated retrieval and manual equivalence selection (IFC 4.3 Dictionary Search):

For each UFGS property, an automated query to the bSDD /api/TextSearch/v2 endpoint (IFC 4.3 dictionary) retrieves candidate IFC property definitions. The user then selects the most appropriate match from the returned candidates, capturing its URI, code, and name.

In [25]:
ifc_props_dict = map_ufgs_to_ifc(dfs, df_names, IFC43)

['compressive strength', 'density', 'weather exposure', 'fire rating', 'mortar type', 'grout compressive strength', 'thermal resistance', 'flexural strength']

STONE WALL CHIMNEY - COMPRESSIVE STRENGTH - CANDIDATE IFC PROPERTIES:
[0] - Compressive Strength: (The compressive strength of the object or material.).
[1] - Raised Compressive Strength: (Alternative value for compressive strength which may be used under material and code dependent conditions (e.g. if deformation is tolerable, or far from ends of the member); conditions should be stated in IfcProperty.[[Description]].).
[2] - None / Not applicable.


Select the index of the best match:  0



STONE WALL CHIMNEY - DENSITY - CANDIDATE IFC PROPERTIES:
[0] - Mass Density: (Material mass density.).
[1] - None / Not applicable.


Select the index of the best match:  0


weather exposure - No IFC PROPERTIES candidates found.

STONE WALL CHIMNEY - FIRE RATING - CANDIDATE IFC PROPERTIES:
[0] - Fire Rating: (Fire rating for this object. It is given according to the national fire safety classification.).
[1] - Fire Resistance Rating: (Measure of the fire resistance rating in hours (e.g., 1.5 hours, 2 hours, etc.).).
[2] - None / Not applicable.


...

water cement ratio - No IFC PROPERTIES candidates found.
maximum aggregate size - No IFC PROPERTIES candidates found.
reinforcement grade - No IFC PROPERTIES candidates found.
reinforcement coating - No IFC PROPERTIES candidates found.

STRIP FOOTING - REINFORCEMENT SPACING - CANDIDATE IFC PROPERTIES:
[0] - Reinforcement Spacing: (The spacing between reinforcing elements.).
[1] - None / Not applicable.


Select the index of the best match:  0


concrete cover to reinforcement - No IFC PROPERTIES candidates found.
curing period - No IFC PROPERTIES candidates found.
['material designation', 'coating type', 'strength grade', 'weld type']
material designation - No IFC PROPERTIES candidates found.
coating type - No IFC PROPERTIES candidates found.

BEAM SHOE - STRENGTH GRADE - CANDIDATE IFC PROPERTIES:
[0] - Strength Grade: (Grade with respect to mechanical strength and stiffness.).
[1] - None / Not applicable.


Select the index of the best match:  0


weld type - No IFC PROPERTIES candidates found.


In [26]:
print(ifc_props_dict)

{'stone_wall_chimney': [{'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/prop/CompressiveStrength', 'code': 'CompressiveStrength', 'name': 'Compressive Strength'}, {'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/prop/MassDensity', 'code': 'MassDensity', 'name': 'Mass Density'}, None, {'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/prop/FireResistanceRating', 'code': 'FireResistanceRating', 'name': 'Fire Resistance Rating'}, None, None, None, None], 'roof_wood_beam': [{'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/prop/BendingStrength', 'code': 'BendingStrength', 'name': 'Bending Strength'}, {'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/prop/TensileStrengthPerp', 'code': 'TensileStrengthPerp', 'name': 'Tensile Strength Perp'}, {'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/prop/CompressiveStrength', 'code': 'CompressiveStrength', 'name': 'Com

### 5.1.2. Properties sets automated retrieval and manual equivalence selection (IFC 4.3 Dictionary Search):

For each IFC property, an automated query to the bSDD /api/Property/Classes/v1 endpoint retrieves the candidate IFC property sets in which the property is defined. The user then selects the most appropriate match from the returned candidates, capturing its URI and code.

In [27]:
ifc_propsets_dict = select_psets_for_properties(ifc_props_dict)


STONE WALL CHIMNEY - COMPRESSIVE STRENGTH - CANDIDATES (PSET):
[0] - Pset_MaterialConcrete: ([[IfcMaterial]] is a homogeneous or inhomogeneous substance that can be used to form elements (physical products or their components).).
[1] - Pset_MechanicalPanelInPlane: ([[IfcMaterial]] is a homogeneous or inhomogeneous substance that can be used to form elements (physical products or their components).).
[2] - Pset_MechanicalPanelOutOfPlane: ([[IfcMaterial]] is a homogeneous or inhomogeneous substance that can be used to form elements (physical products or their components).).
[3] - Pset_MechanicalPanelOutOfPlaneNegative: ([[IfcMaterial]] is a homogeneous or inhomogeneous substance that can be used to form elements (physical products or their components).).
[4] - Pset_MaterialConcrete: (A set of extended mechanical properties related to concrete materials.).
[5] - Pset_MechanicalPanelInPlane: (Properties for Mechanical Panels In Plane.).
[6] - Pset_MechanicalPanelOutOfPlane: (Properties fo

Select the index of the best match:  8



STONE WALL CHIMNEY - MASS DENSITY - CANDIDATES (PSET):
[0] - Pset_MaterialCommon: ([[IfcMaterial]] is a homogeneous or inhomogeneous substance that can be used to form elements (physical products or their components).).
[1] - Pset_MaterialCommon: (A set of general material properties.).
[2] - None / Not applicable.


Select the index of the best match:  1



STONE WALL CHIMNEY - FIRE RESISTANCE RATING - CANDIDATES (PSET):
[0] - Pset_DamperTypeFireDamper: (Fire damper used to prevent the spread of fire for a specified duration. Commonly operated by fusable link that melts above a certain temperature.).
[1] - Pset_DamperTypeFireSmokeDamper: (Combination fire and smoke damper used to prevent the spread of fire and smoke. Commonly operated by a fusable link and a smoke detector.).
[2] - Pset_DamperTypeFireDamper: (Fire damper type attributes.; Pset renamed from Pset_DamperTypeFire to [[Pset_DamperTypeFireDamper]] in IFC2x2 Pset Addendum.).
[3] - Pset_DamperTypeFireSmokeDamper: (Combination Fire and Smoke damper type attributes.; New Pset in IFC2x2 Pset Addendum.).
[4] - None / Not applicable.


...

Select the index of the best match:  4



STRIP FOOTING - REINFORCEMENT SPACING - CANDIDATES (PSET):
[0] - Pset_DuctSegmentTypeCommon: (A duct segment is used to typically join two sections of duct network.).
[1] - Pset_DuctSegmentTypeCommon: (A flexible segment is a continuous non-linear segment of duct that can be deformed and change the direction of flow.).
[2] - Pset_DuctSegmentTypeCommon: (Duct segment type common attributes.).
[3] - Pset_DuctSegmentTypeCommon: (A rigid segment is a continuous linear segment of duct that cannot be deformed.).
[4] - None / Not applicable.


Select the index of the best match:  4



BEAM SHOE - STRENGTH GRADE - CANDIDATES (PSET):
[0] - Pset_MaterialWood: ([[IfcMaterial]] is a homogeneous or inhomogeneous substance that can be used to form elements (physical products or their components).).
[1] - Pset_MaterialWood: (This is a collection of properties applicable to wood-based materials that specify kind and grade of material as well as moisture related parameters.).
[2] - None / Not applicable.


Select the index of the best match:  2


In [28]:
print(ifc_propsets_dict)

{'stone_wall_chimney': [None, {'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/class/Pset_MaterialCommon', 'code': 'Pset_MaterialCommon', 'name': 'Property Set: Material Common'}, None, None, None, None, None, None], 'roof_wood_beam': [{'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/class/Pset_MechanicalBeamOutOfPlane', 'code': 'Pset_MechanicalBeamOutOfPlane', 'name': 'Property Set: Mechanical Beam Out Of Plane'}, {'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/class/Pset_MechanicalBeamOutOfPlane', 'code': 'Pset_MechanicalBeamOutOfPlane', 'name': 'Property Set: Mechanical Beam Out Of Plane'}, None, None, {'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/class/Pset_MechanicalBeamOutOfPlane', 'code': 'Pset_MechanicalBeamOutOfPlane', 'name': 'Property Set: Mechanical Beam Out Of Plane'}, None, None, None], 'strip_footing': [{'uri': 'https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/cl

## 5.2. Mapping UFGS Technical properties to ETIM/bsDD:

#### ETIM/bsDD stage: property and class mapping for IFC gaps:

This stage supplements the IFC pass by filling gaps with ETIM terms. It proceeds in two steps, both kept index-aligned with the original UFGS parameter list (positions that already have IFC matches remain None):

- **ETIM property lookup:**
For each UFGS property marked None after the IFC 4.3 pass, query bSDD /api/TextSearch/v2 restricted to ETIM (TypeFilter=Property). Select the best match and record {uri, code, name} into etim_props_dict.

- **ETIM class lookup:**
For each ETIM property in etim_props_dict, query /api/Property/Classes/v1 with the property’s URI to retrieve ETIM classes that reference it. Select the most relevant class per property and record {uri, code (derived from the class URI), name} (or None if unsuitable).

Note: ETIM organizes properties under classes (not IFC-style property sets).

### 5.2.1. Properties ETIM/bsDD search for IFC gaps:

For UFGS properties with no IFC equivalent (indices marked None after the IFC 4.3 pass), query bSDD /api/TextSearch/v2 restricted to ETIM (TypeFilter=Property). Select the best match and record its URI, code, and name; when needed, call /api/Property/v1 with the selected PropertyUri to list related ETIM classes. Store results in etim_props_dict, index-aligned to the UFGS list (positions with IFC matches remain None). Note: ETIM organizes properties under classes, not IFC-style property sets.

In [29]:
etim_props_dict = select_etim_props_for_ifc_gaps(dfs, df_names, ifc_props_dict)

stone_wall_chimney
compressive strength - IFC equivalent property already found.
density - IFC equivalent property already found.
weather exposure - No ETIM PROPERTIES candidates found.
fire rating - IFC equivalent property already found.

STONE WALL CHIMNEY - MORTAR TYPE - CANDIDATE ETIM PROPERTIES:
[0] - Masonry mortar type according to EN 998-2: ().
[1] - Type of mortar: ().
[2] - None / Not applicable.


Select the index of the best match:  0


grout compressive strength - No ETIM PROPERTIES candidates found.

STONE WALL CHIMNEY - THERMAL RESISTANCE - CANDIDATE ETIM PROPERTIES:
[0] - Thermal resistance value: ().
[1] - None / Not applicable.


Select the index of the best match:  0



STONE WALL CHIMNEY - FLEXURAL STRENGTH - CANDIDATE ETIM PROPERTIES:
[0] - Flexural strength according to EN 12808-3: ().
[1] - Flexural strength according to EN 13813: ().
[2] - Flexural strength according to EN 998-1: (The flexural and compressive strength of moulded mortar specimens.).
[3] - None / Not applicable.


...

roof_wood_beam
bending strength - IFC equivalent property already found.
tensile strength - IFC equivalent property already found.
compressive strength - IFC equivalent property already found.

ROOF WOOD BEAM - MODULUS OF ELASTICITY - CANDIDATE ETIM PROPERTIES:
[0] - Modulus of elasticity: ().
[1] - None / Not applicable.


Select the index of the best match:  0


shear strength - IFC equivalent property already found.
fire rating - IFC equivalent property already found.
preservative retention - No ETIM PROPERTIES candidates found.
durability - IFC equivalent property already found.
strip_footing
compressive strength - IFC equivalent property already found.
water cement ratio - No ETIM PROPERTIES candidates found.
maximum aggregate size - No ETIM PROPERTIES candidates found.
reinforcement grade - No ETIM PROPERTIES candidates found.
reinforcement coating - No ETIM PROPERTIES candidates found.
reinforcement spacing - IFC equivalent property already found.
concrete cover to reinforcement - No ETIM PROPERTIES candidates found.
curing period - No ETIM PROPERTIES candidates found.
beam_shoe
material designation - No ETIM PROPERTIES candidates found.

BEAM SHOE - COATING TYPE - CANDIDATE ETIM PROPERTIES:
[0] - Type of coating: ().
[1] - None / Not applicable.


Select the index of the best match:  0


strength grade - IFC equivalent property already found.
weld type - No ETIM PROPERTIES candidates found.


In [30]:
print(etim_props_dict)

{'stone_wall_chimney': [None, None, None, None, {'uri': 'https://identifier.buildingsmart.org/uri/etim/etim/10.0/prop/EF010452', 'code': 'EF010452', 'name': 'Masonry mortar type according to EN 998-2'}, None, {'uri': 'https://identifier.buildingsmart.org/uri/etim/etim/10.0/prop/EF010245', 'code': 'EF010245', 'name': 'Thermal resistance value'}, {'uri': 'https://identifier.buildingsmart.org/uri/etim/etim/10.0/prop/EF016901', 'code': 'EF016901', 'name': 'Flexural strength according to EN 998-1'}], 'roof_wood_beam': [None, None, None, {'uri': 'https://identifier.buildingsmart.org/uri/etim/etim/10.0/prop/EF026335', 'code': 'EF026335', 'name': 'Modulus of elasticity'}, None, None, None, None], 'strip_footing': [None, None, None, None, None, None, None, None], 'beam_shoe': [None, {'uri': 'https://identifier.buildingsmart.org/uri/etim/etim/10.0/prop/EF024707', 'code': 'EF024707', 'name': 'Type of coating'}, None, None]}


### 5.2.2. ETIM/bsDD property-class lookup for IFC gaps:

For UFGS parameters without an IFC match, each ETIM property in etim_props_dict is queried via bSDD /api/Property/Classes/v1 to retrieve ETIM classes that reference it. Using select_etim_classes_for_properties, the most relevant class is selected per property. The output records {uri, code (derived from the class URI), name}, or None if no suitable class is identified, index-aligned with the original UFGS parameter list.

In [31]:
etim_classes_dict = select_etim_classes_for_properties(etim_props_dict)


STONE WALL CHIMNEY - MASONRY MORTAR TYPE ACCORDING TO EN 998-2 - CANDIDATES (ETIM):
[0] - EC003182 — Brickwork/pointing mortar (A specialized mortar used for bonding and finishing brick joints.)
[1] - None / Not applicable.


Select the index of the best match:  0



STONE WALL CHIMNEY - THERMAL RESISTANCE VALUE - CANDIDATES (ETIM):
[0] - EC003225 — Filling element system floor (A material used to fill gaps or create a leveling base in flooring systems.)
[1] - EC003226 — Floor joist system floor (A structural framework of joists that supports a floor.)
[2] - EC003106 — Roof element pitched roof (A prefabricated structural component designed for sloped roofs.)
[3] - None / Not applicable.


Select the index of the best match:  3



STONE WALL CHIMNEY - FLEXURAL STRENGTH ACCORDING TO EN 998-1 - CANDIDATES (ETIM):
[0] - EC004679 — Rendering and plastering mortar (A type of mortar used for coating and finishing walls and surfaces.)
[1] - None / Not applicable.


Select the index of the best match:  1



ROOF WOOD BEAM - MODULUS OF ELASTICITY - CANDIDATES (ETIM):
[0] - EC003107 — Facade plate (A protective or decorative panel used for exterior building facades.)
[1] - None / Not applicable.


Select the index of the best match:  1



BEAM SHOE - TYPE OF COATING - CANDIDATES (ETIM):
[0] - EC010187 — Adjustment system washbasin (A mechanism that allows for height or angle adjustments of a washbasin.)
[1] - EC011026 — Auxiliary support leg set disabled sanitary (A set of additional support legs for sanitary installations designed for disabled users.)
[2] - EC010948 — Backrest (A support structure designed to provide back support.)
[3] - EC010575 — Bath seat (A seat placed in a bathtub for comfort and safety.)
[4] - EC003065 — Bishop cap (A decorative stone or concrete element used on walls or fences.)
[5] - EC003232 — Bond stone (A specially shaped stone used to interlock masonry structures.)
[6] - EC011737 — Channel drain (A surface drainage system designed to manage rainwater or wastewater flow.)
[7] - EC012349 — Concrete tile (A tile made from concrete used for flooring or roofing.)
[8] - EC003062 — Concrete tile machine package (A set of machines and tools used to manufacture concrete tiles.)
[9] - EC011428 — Cor

Select the index of the best match:  35


In [32]:
print(etim_classes_dict)

{'stone_wall_chimney': [None, None, None, None, {'uri': 'https://identifier.buildingsmart.org/uri/etim/etim/10.0/class/EC003182', 'code': 'EC003182', 'name': 'Brickwork/pointing mortar'}, None, None, None], 'roof_wood_beam': [None, None, None, None, None, None, None, None], 'strip_footing': [None, None, None, None, None, None, None, None], 'beam_shoe': [None, None, None, None]}


# 6. Fill bsDD datasets columns:

#### DEVELOPMENT PHASE: Dictionaries SAVE.

In [33]:
# payload = {
#     "ifc_props_dict": ifc_out,
#     "ifc_propsets_dict": ifc_propsets_dict,
#     "etim_props_dict": etim_props_dict,
#     "etim_classes_dict": etim_classes_dict,
# }

# with open("bsdd_cache.json", "w", encoding="utf-8") as f:
#     json.dump(payload, f, ensure_ascii=False, indent=2)

#### DEVELOPMENT PHASE: Dictionaries LOAD.

In [34]:
# with open("bsdd_cache.json", "r", encoding="utf-8") as f:
#     d = json.load(f)

# ifc_props_dict    = d["ifc_props_dict"]
# ifc_propsets_dict = d["ifc_propsets_dict"]
# etim_props_dict   = d["etim_props_dict"]
# etim_classes_dict = d["etim_classes_dict"]

In [35]:
apply_property_mapping(dfs, df_names, ifc_props_dict, col_prefix="ifc_property", exists_col="ifc_exists")

In [36]:
apply_property_mapping(dfs, df_names, ifc_propsets_dict, col_prefix="ifc_pset")

In [37]:
apply_property_mapping(dfs, df_names, etim_props_dict,   col_prefix="etim_property", exists_col="etim_exists")

In [38]:
apply_property_mapping(dfs, df_names, etim_classes_dict, col_prefix="etim_class")

# 7. Global unique identifier (GUID) generation and missing IFC Pset and ETIM Class placeholders:

**uuid5** is used to generate deterministic, reproducible identifiers based on input data rather than random values. It derives each GUID from a fixed namespace and the element’s defining attributes, ensuring that identical inputs always produce the same identifier. This approach maintains consistency, traceability, and uniqueness across repeated processing or dataset updates.  

In [103]:
dfs = df_guid_placeholders(dfs)

In [104]:
out = guid_check(dfs)

STONE WALL CHIMNEY has no GUID duplicate value: True.
ROOF WOOD BEAM has no GUID duplicate value: True.
STRIP FOOTING has no GUID duplicate value: True.
BEAM SHOE has no GUID duplicate value: True.


# 8. Data save:

In [105]:
output_path = "../data/processed/bsdd_collecting.xlsx"

In [106]:
# output_path2 = "../data/processed/bsdd_collecting2.xlsx"

In [107]:
write_multi_df_to_excel(dfs, output_path)